_Goal Here is to train the model, similar to previous notebooks, but without using Trainer class_

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from datasets import load_dataset 
from transformers import AutoTokenizer, DataCollatorWithPadding

# Loading Dataset
raw_dataset = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Preprocessing Dataset
def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)

tokenized_datasets = raw_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [3]:
# Pos processing Dataset

tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"]) # Model Doesn't need these columns
tokenized_datasets = tokenized_datasets.rename_column("label", "labels") # Model expects label column to be named `labels`
tokenized_datasets.set_format("torch")  # PyTorch is used for training

In [4]:
# checking the dataset
tokenized_datasets.column_names

{'train': ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
 'validation': ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
 'test': ['labels', 'input_ids', 'token_type_ids', 'attention_mask']}

In [5]:
# Defining the DataLoaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator)

In [6]:
for batch in train_dataloader:
    print({k: v.shape for k, v in batch.items()})   # Printing shape of ids, attention_mask, labels for each batch

{'labels': torch.Size([8]), 'input_ids': torch.Size([8, 72]), 'token_type_ids': torch.Size([8, 72]), 'attention_mask': torch.Size([8, 72])}
{'labels': torch.Size([8]), 'input_ids': torch.Size([8, 98]), 'token_type_ids': torch.Size([8, 98]), 'attention_mask': torch.Size([8, 98])}
{'labels': torch.Size([8]), 'input_ids': torch.Size([8, 70]), 'token_type_ids': torch.Size([8, 70]), 'attention_mask': torch.Size([8, 70])}
{'labels': torch.Size([8]), 'input_ids': torch.Size([8, 59]), 'token_type_ids': torch.Size([8, 59]), 'attention_mask': torch.Size([8, 59])}
{'labels': torch.Size([8]), 'input_ids': torch.Size([8, 62]), 'token_type_ids': torch.Size([8, 62]), 'attention_mask': torch.Size([8, 62])}
{'labels': torch.Size([8]), 'input_ids': torch.Size([8, 81]), 'token_type_ids': torch.Size([8, 81]), 'attention_mask': torch.Size([8, 81])}
{'labels': torch.Size([8]), 'input_ids': torch.Size([8, 77]), 'token_type_ids': torch.Size([8, 77]), 'attention_mask': torch.Size([8, 77])}
{'labels': torch.Siz

In [7]:
# defining the model

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor(0.7582, grad_fn=<NllLossBackward0>) torch.Size([4, 2])


In [8]:
# setting up the optimizer and scheduler
from transformers import AdamW 
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)

optimizer = AdamW(model.parameters(), lr=5e-5)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_training_steps=num_training_steps,
    num_warmup_steps=0
)

print(num_training_steps)

1377


In [9]:
# setting up the training loop

import torch 

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
print(f"Using Device: {device}")

Using Device: cuda


In [10]:
# the training loop

from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

100%|██████████| 1377/1377 [03:05<00:00,  7.90it/s]

In [11]:
# evaluating the model

import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8431372549019608, 'f1': 0.8915254237288136}